In [7]:
# prompt: mount

from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


## Import Packages

In [3]:
import platform

print("Running on Windows")
#path_main_folder = "/content/drive/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"
path_main_folder = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/"


path_data_folder = path_main_folder + "AIST_Cleaned/data3_san_stefano/"#"AIST_Cleaned/data2_mosheerIsmail/",data3_san_stefano
path_cfg = path_data_folder +"cfg.sumocfg"
path_project_folder = path_main_folder + ""



Running on Windows


In [2]:
import os
os.path.exists(path_project_folder)

NameError: name 'path_project_folder' is not defined

In [4]:
import sys
sys.path.append(path_project_folder)

from Connections.Proto_Connection_Utils.ardinuo import ardinuo
a = ardinuo(4,"COM3")

In [7]:
a.send_command("y",4)

In [4]:
import sys
sys.path.append(path_project_folder)

from Connections.ProtoRealConnection import status_acc,status_chk,status_emr,status_free
import Connections.ProtoRealConnection
from Connections.Connection import *
import os
import gymnasium as gym
from numpy import inf
import numpy as np
import SumoEnvSingleAgent
from Utils_reporting import *
from Utils_running_singleAgent import *
from rewards import *
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
import torch
import time
from Callbacks import *
from models.d3qn import D3QNAgent

## Load Parameters

In [5]:
durations = [7,10,12,15] #[15,30,60,90]
ENV_NAME = "HighGroupedSumoEnv"
REWARD_TYPE = "literature"
max_steps = 100000000 
max_sumo_steps = 2000
sumo_traffic_scale = 1 #.14
enable_variation_action = True
seed = 3



In [6]:
agent_id ='6082411793'
traffic_lights = [(4, agent_id)]


In [7]:
reward_func = {
'proposed_reward':reward_proposed,
'literature':reward_liter,
'project_reward':reward_proj,
}
env_classes = {
    "HighGroupedSumoEnv": SumoEnvSingleAgent.HighGroupedSumoEnv,
    "GroupedSumoEnv": SumoEnvSingleAgent.GroupedSumoEnv ,
    "SumoEnv": SumoEnvSingleAgent.SumoEnv,
}

In [8]:
def create_env(config_):
    args = (traffic_lights[0][1],traffic_lights[0][0], durations,reward_func[REWARD_TYPE], max_steps, max_sumo_steps,sumo_traffic_scale,enable_variation_action,config_,seed)
    env = env_classes[ENV_NAME](*args)
    return env



In [9]:
yolo_path = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/try/yolo11n.pt"
yaml_file = "I:/My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager/config.yaml"
path_model='FINAL_HighGroupedSumoEnv_literature_exp217.keras'

thresould_speed = .01 #10 ,36 is very wrong as m/s #.01 m/s
yellow_seconds = 15

In [10]:
ardinuo_port = 'COM3'
global_url="http://127.0.0.1:8000"
broker="broker.hivemq.com"
mqtt_port= 1883

## Load Parameters for model

In [11]:
import yaml


with open(yaml_file, "r") as file:
    config = yaml.safe_load(file)


In [12]:
general_settings = config['general_settings']
experiment_settings_changable = config["experiment_settings"]['changable_settings']
experiment_settings_const = config["experiment_settings"]["const_settings"]
algorithm_settings=config["algorithms_settings"]

d3qn_Settings = algorithm_settings['D3QN']
d3qn_experiment_settings =d3qn_Settings[str(ENV_NAME+"_"+REWARD_TYPE)]


parameters={'learning_rate': d3qn_experiment_settings['learning_rate'],
 'gamma': d3qn_experiment_settings['gamma'],
 'tau': d3qn_experiment_settings['tau'],
 'l2_reg': d3qn_experiment_settings['l2_reg'],
 'epsilon_decay': d3qn_experiment_settings['epsilon_decay'],
 'batch_size': d3qn_experiment_settings['batch_size']}


## Make env

In [13]:
import importlib
import Connections.ProtoRealConnection
importlib.reload(Connections.ProtoRealConnection)


<module 'Connections.ProtoRealConnection' from 'I:\\My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager\\Connections\\ProtoRealConnection.py'>

In [13]:
conn = Connections.ProtoRealConnection.ProtoRealConnection(ardinuo_port,mqtt_port,agent_id,global_url,broker,yolo_path,yellow_seconds,thresould_speed = thresould_speed)

set_global_conn(conn)


AssertionError: Error reading video file

In [14]:
env = create_env({})  # Create the environment instance
vec_env = DummyVecEnv([lambda: env])
vec_env = VecNormalize(vec_env, norm_obs=True, norm_reward=False)


vec_env.reset()


------------ STARTING MQTT LISTENER AT state/6082411793/reply ------------
------------ STARTING MQTT PUBLISHER AND LISTENER AT state/6082411793/msg------------
Environment reset


I:\My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager\Connections\ProtoRealConnection.py:249: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client()


array([[          0,           0,           0,           0,           0,           0,           0]], dtype=float32)

📥 Received: QL 10
📥 Received: CHK
📥 Received: AVBL
📥 Received: AVBL
📥 Received: DONE QL?
📥 Received: FREE
📥 Received: QL 10
📥 Received: DONE QL?
📥 Received: QL 10
📥 Received: EMR
📥 Received: QL 10
📥 Received: QL 10
📥 Received: EMR 100
📥 Received: EMR 100
📥 Received: EMR 100
📥 Received: EMR 100
📥 Received: FREE


In [15]:
agent = D3QNAgent(
            env=vec_env,
            state_size=vec_env.observation_space.shape,
            num_actions=vec_env.action_space.n,
            memory_size=100000,
            batch_size=parameters['batch_size'],
            gamma=parameters['gamma'],
            epsilon_start=1.0,
            epsilon_min=0.01,
            epsilon_decay=parameters['epsilon_decay'],
            learning_rate=parameters['learning_rate'],
            tau=parameters['tau'],
            update_freq=4,
            l2_reg=parameters['l2_reg'],
            random_state=seed
        )




Environment observation space: Box(-inf, inf, (7,), float32)
Environment action space: Discrete(8)
State size provided: (7,)


In [16]:
import keras

keras.config.enable_unsafe_deserialization()
agent.load_model(path_model)


In [17]:
env.encoded_action_mapping

{0: ('rrrr', 7),
 1: ('rrrr', 10),
 2: ('rrrr', 12),
 3: ('rrrr', 15),
 4: ('gggg', 7),
 5: ('gggg', 10),
 6: ('gggg', 12),
 7: ('gggg', 15)}

In [18]:
conn.yellow_seconds = 7 #7,10,12,15

In [19]:
import threading
import time
import requests
peirod_of_send = 13 # longer for deploy
def post_traffic_data():
    while True:
        
        try:
            payload = {
                "speed": conn.getCurrentState(agent_id)[0],
                "var_speed": conn.getCurrentState(agent_id)[1],
                "waiting_time": conn.getCurrentState(agent_id)[2],
                "var_waiting": conn.getCurrentState(agent_id)[3],
                "avg_throughput": conn.getCurrentState(agent_id)[4],
                "avg_queue_length": conn.getCurrentState(agent_id)[5],
                "avg_occupancy": conn.getCurrentState(agent_id)[6],
                "reward": str(reward_func[REWARD_TYPE](agent_id,conn.getCurrentState(agent_id),None,None)),
                "last_action": str(0), # for now not store them
                "action": str(0) # for now not store them
            }

            response = requests.post("https://taha454-trafficmanager-account.hf.space/traffic-data/add", json=payload)

            if response.ok:
                print("Response JSON:", response.json() )
            else:
                print("Error:", response.status_code, response.text)

        except Exception as e:
            print("Error during traffic data post:", e)

        time.sleep(peirod_of_send)  # Sleep 30 seconds before next post

# Start the thread (daemon=True to auto-exit when main thread exits)
threading.Thread(target=post_traffic_data, daemon=True).start()


I:\My Drive/study/graduation_project/final/Code/project_files/TrafficManager/TrafficManager\rewards.py:78: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if not road_inf:  # This covers None or an empty list


Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response JSON: {'message': 'Traffic data added successfully'}
Response

In [20]:
import random
import serial


while True:
    print("global_message: ", conn.global_message)
    if conn.global_message==status_free:
        predicated_action = agent.get_action(conn.getCurrentState(agent_id))

        action,duration = env.encoded_action_mapping[predicated_action]
        print(f"ACTION {predicated_action} Signal {action} with duration {duration} seconds")
 
        conn.is_rl = True
        vec_env.step([predicated_action])


    elif conn.global_message.startswith(status_emr) or conn.global_message.startswith(status_acc):
        
        type_r,duration=conn.global_message.split()
        action = 'g' if type_r=='EMR' else 'r'
        print("start_action _  ",action," ",type_r," ",str(duration))
        
        conn.is_rl = False
        conn.ardinuo.reset()
        time.sleep(1)  # Wait for ardinuo to reset
        conn.do_steps_duration(action,int(duration),0,0,0)
        conn.is_rl =True



    
    


global_message:  FREE
ACTION 7 Signal gggg with duration 15 seconds
Progress: 1/2000, Sumo Time 0, state [     8.8194      73.664           0           0         100           0          40], action gggg, duration 15, reward 0.0global_message:  FREE
ACTION 7 Signal gggg with duration 15 seconds
Progress: 2/2000, Sumo Time 0, state [        7.5        68.3           0           0         300           0         100], action gggg, duration 15, reward 0.0global_message:  FREE
ACTION 2 Signal rrrr with duration 12 seconds
Progress: 3/2000, Sumo Time 0, state [        7.5        68.3           0           0         300           0         100], action rrrr, duration 12, reward 0.0global_message:  EMR 100
start_action _   g   EMR   100
global_message:  FREE
ACTION 2 Signal rrrr with duration 12 seconds


SerialException: WriteFile failed (PermissionError(13, 'The device does not recognize the command.', None, 22))